In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [3]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [4]:
import os
from dotenv import load_dotenv
env_path = "templates/app_schema/.env"
if os.path.exists(env_path):
    load_dotenv(env_path)
    print(f"Loaded environment variables from {env_path}")

Loaded environment variables from templates/app_schema/.env


In [5]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [7]:
from datetime import datetime
from server import Prompt

user_prompts = Prompt("""Bot that says hello""".strip(), datetime.now(), "user")        

In [9]:

my_bot = application.prepare_bot([user_prompts], bot_id="handler_tests", capabilities=["perplexity.web_search"])
my_bot.__dict__

Compiling TypeSpec...


{'refined_description': None,
 'capabilities': CapabilitiesOut(capabilities=['perplexity.web_search'], error_output=None),
 'typespec': TypespecOut(reasoning='Based on the minimal user request "Bot that says hello", this appears to be a simple greeting bot that responds to user greetings. Since there\'s very limited context, I\'ll create a basic interface with just enough functionality to handle greetings.\n\nThe main function will be to greet the user, which would take input like user\'s name and time of day to generate an appropriate greeting.\n\nPossible scenarios:\n1. Basic greeting - User says "hello" and bot responds\n2. Personalized greeting - User introduces themselves and bot greets them by name\n3. Time-specific greeting - Bot uses time of day to say "good morning", "good afternoon", etc.\n\nI\'ll create these models and interface with appropriate scenarios:', typespec_definitions='model GreetingOptions {\n  userName?: string;\n  timeOfDay?: string;\n}\n\nmodel PersonalizedGr

In [ ]:
my_bot = application.prepare_bot([user_prompts, Prompt("Add say goodbye", datetime.now(), "user")], bot_id="handler_tests", capabilities=["perplexity.web_search"])
my_bot.__dict__

Compiling TypeSpec...


In [12]:
my_bot = application.update_bot(
    typespecSchema=my_bot.typespec.typespec_definitions, 
    bot_id="web_search_bot", 
    capabilities=["perplexity.web_search"]
)

Creating prompt from TypeSpec...
Processing TypeSpec...
Generating Typescript Schema...
Generating Drizzle Schema...
Generating Handler Tests...
Generating Handlers...


In [14]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root

In [15]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='model HelloRequest {\n  name: string;\n}\n\nmodel GoodbyeRequest {\n  name: string;\n}\n\ninterface GreetingBot {\n  @scenario("""\n  Scenario: Basic greeting\n  When the user says "Hello" or "Hi"\n  Then the system should respond with a greeting\n  \n  Examples:\n    | Input                 | Expected Name |\n    | "Hello"               | undefined     |\n    | "Hi, I\'m Jane"        | "Jane"        |\n    | "Hello, my name is Bob" | "Bob"      |\n  """)\n  @scenario("""\n  Scenario: Greeting with time of day\n  When the user greets with a time reference\n  Then the system should incorporate the time in response\n  \n  Examples:\n    | Input                | Expected Greeting        |\n    | "Good morning"       | Morning-specific greeting |\n    | "Hey, good evening"  | Evening-specific greeting |\n  """)\n  @llm_func("Generate a personalized greeting based on user input")\n  sayHello(options: HelloRequest): void;\n\n  @scenario(